In [3]:
import torch
from ordered_set import OrderedSet
from collections import defaultdict as ddict

In [5]:
ent_set, rel_set = OrderedSet(), OrderedSet()
data_dir = './'
dataset = 'FB15k'
triplet_no_reverse=True

ent_set, rel_set = OrderedSet(), OrderedSet()


for split in ['train', 'test', 'valid']:
    for line in open(data_dir+'/{}/{}.txt'.format(dataset, split)):
        # print(line.strip().split('\t'))
        if len(line.strip().split('\t')) < 3:
            
            continue
        sub, rel, obj = map(str.lower, line.strip().split('\t'))
        

        ent_set.add(sub)
        rel_set.add(rel)
        ent_set.add(obj)

ent2id = {ent: idx for idx, ent in enumerate(ent_set)}
rel2id = {rel: idx for idx, rel in enumerate(rel_set)}

rel2id.update({rel+'_reverse': idx+len(rel2id) for idx, rel in enumerate(rel_set)})

id2ent = {idx: ent for ent, idx in ent2id.items()}
id2rel = {idx: rel for rel, idx in rel2id.items()}

num_ent		= len(ent2id)
num_rel		= len(rel2id) // 2
# embed_dim	= k_w * k_h if embed_dim is None else embed_dim

print('dataset: ', dataset)
print('number of entities: ', num_ent)
print('number of relations: ', num_rel)


data = ddict(list)
sr2o = ddict(set)
train_edge_num = 0
unique_train = set()
for split in ['train', 'test', 'valid']:
    for line in open(data_dir+'/{}/{}.txt'.format(dataset, split)):
        if len(line.strip().split('\t')) < 3:
            continue
        sub, rel, obj = map(str.lower, line.strip().split('\t'))

        sub, rel, obj = ent2id[sub], rel2id[rel], ent2id[obj]
    

        data[split].append((sub, rel, obj))

        if split == 'train': 
            train_edge_num += 1
            unique_train.add(sub)
            unique_train.add(obj)

            sr2o[(sub, rel)].add(obj) 

            if not triplet_no_reverse:
                # print('use inverse in loss')
                sr2o[(obj, rel+num_rel)].add(sub)
            else:
                # print('do not use inverse in loss')
                sr2o[(obj, rel)].add(sub)


data = dict(data)

sr2o = {k: list(v) for k, v in sr2o.items()}
for split in ['test', 'valid']:
    for sub, rel, obj in data[split]:
        sr2o[(sub, rel)].add(obj)

        if not triplet_no_reverse:
            sr2o[(obj, rel+num_rel)].add(sub)
        else:
            sr2o[(obj, rel)].add(sub)


sr2o_all = {k: list(v) for k, v in sr2o.items()}

triples  = ddict(list)


for (sub, rel), obj in sr2o.items():
    # num_label += len(sr2o[(sub, rel)])
    triples['train'].append({'triple':(sub, rel, obj[0]), 'label': sr2o[(sub, rel)],'sub_samp': 1})


for split in ['test', 'valid']:

    for sub, rel, obj in data[split]:
        if not triplet_no_reverse:
            rel_inv = rel + num_rel
        else:
            rel_inv = rel
    
        
        triples['{}_{}'.format(split, 'tail')].append({'triple': (sub, rel, obj), 	   'label': sr2o_all[(sub, rel)]})
        triples['{}_{}'.format(split, 'head')].append({'triple': (obj, rel_inv, sub), 'label': sr2o_all[(obj, rel_inv)]})

triples = dict(triples)

    

dataset:  FB15k
number of entities:  14951
number of relations:  1345


AttributeError: 'list' object has no attribute 'add'